In [ ]:
!pip3 install -e /Users/gva/Documents/PhD/workspace/sgd/

In [ ]:
from sgd import *
import time
import statsmodels.api as sm
# from sklearn.preprocessing import normalize

In [ ]:
n, p = int(1e+5), int(1e+1)

In [ ]:
X = np.empty((n, p), dtype=np.float64)
X = np.random.normal(0, 1, size=(n, p))
# X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
X = np.hstack((np.ones((n, 1)), X)) ## add intercept

In [ ]:
theta = np.ones(p+1) * 0.4

In [ ]:
## Gaussian
# eps = np.random.normal(0, 1, size=n)
# Y = X @ theta + eps

In [ ]:
## Poisson
eta = np.exp(X @ theta)
poisson = np.vectorize(lambda x: np.random.poisson(x, 1))
Y = poisson(eta)

In [ ]:
D = data_set(X, Y)

In [ ]:
familia = sm.families.Poisson(link=sm.families.links.log())

glm_bnch = sm.GLM(D._Y, D._X, family = familia)
true_est = glm_bnch.fit()
# true_est.params

In [ ]:
true_est.params

In [ ]:
m = glm(0.0, 0.0, "poisson", "exponential")

In [ ]:
timer = time
# n: number of observations
# p: number of features
n, p = D._X.shape

details = {
    "method": "implicit",
    "nparams": p,
    "reltol": 1e-32,
    "npasses": 20,
    "size": 10,
    "pass": True,
    "check": True,
    "truth": theta
}

if details["method"] == "implicit": tester = ImplicitSGD(n, timer, **details)
elif details["method"] == "explicit": tester = ExplicitSGD(n, timer, **details)

In [ ]:
n_passes = tester.get_value_of("n_passes")
good_gradient = True
good_validity = True
averaging = False

In [ ]:
theta_old = np.ones(p) / 100.0
theta_old_ave = theta_old
max_iters = n * n_passes
do_more_iterations = True
converged = False

t = 1

while do_more_iterations:
    theta_new = tester.update(t, theta_old, D, m, good_gradient)
    if not averaging:
        tester.sync_members(theta_new)
        converged = tester.convergence(theta_new, theta_old)
    else:
        theta_new_ave = (1.0 - 1.0/float(t)) * theta_old_ave + (1.0/float(t)) * theta_new
        tester.sync_members(theta_new_ave)
        converged = tester.convergence(theta_new_ave, theta_old_ave)
        theta_old_ave = theta_new_ave
    if converged: do_more_iterations = False
    theta_old = theta_new
    if t == max_iters: do_more_iterations = False
    t += 1

In [ ]:
theta_new

In [ ]:
converged

#### Data from file

In [ ]:
# from numpy import genfromtxt

In [ ]:
# data = genfromtxt('glm_test.csv', delimiter=',', encoding='UTF-8', skip_header=1)

In [ ]:
# D = data_set(data[:,1:], data[:,0])

In [ ]:
#data = sm.datasets.scotland.load()
#data.exog = sm.add_constant(data.exog, prepend=False)
#D = data_set(data.exog, data.endog)

In [ ]:
#data = sm.datasets.scotland.load(as_pandas=True)
#data.exog = sm.add_constant(data.exog, prepend=False)
#data.exog["RESPONSE"] = data.endog
#data.exog.to_csv(os.path.join("~/Documents/PhD/workspace/target/misc/glm_test.csv"), index=False, encoding='UTF-8')